In [ ]:

# -*- coding: utf-8 -*-
"""Sentiment Analysis with BERT/RoBERTa using Hugging Face Transformers"""

# Install the required libraries if not already installed
# !pip install transformers torch pandas scikit-learn


In [ ]:

# Import necessary libraries
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:

# Load data
df_train = pd.read_csv('../Dataset/Train.csv')
df_test = pd.read_csv('../Dataset/Test.csv')
df_valid = pd.read_csv('../Dataset/Valid.csv')

# Check the shape of the data
print(df_train.shape, df_test.shape, df_valid.shape)


In [ ]:

# Choose a model (e.g., BERT or RoBERTa)
model_name = 'bert-base-uncased'  # Change to 'roberta-base' for RoBERTa
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:

# Prepare the data for training and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(df_train['text'], df_train['label'], test_size=0.2, random_state=42)
test_texts = df_test['text']
test_labels = df_test['label']

# Tokenize the texts
train_encodings = tokenizer(train_texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(train_labels.values)
val_labels = torch.tensor(val_labels.values)
test_labels = torch.tensor(test_labels.values)


In [ ]:

# Create the training and validation datasets
train_dataset = {'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': train_labels}
val_dataset = {'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask'], 'labels': val_labels}
test_dataset = {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'labels': test_labels}


In [ ]:

# Define the metrics function for evaluation inline
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


In [ ]:

# Initialize the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


In [ ]:

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Validation Results: {eval_results}")


In [ ]:

# Test the model on the test set
test_results = trainer.predict(test_dataset)
print(f"Test Results: {test_results.metrics}")


In [ ]:

# Display the evaluation metrics
print("Test Accuracy:", test_results.metrics['test_accuracy'])
print("Test F1 Score:", test_results.metrics['test_f1'])
print("Test Precision:", test_results.metrics['test_precision'])
print("Test Recall:", test_results.metrics['test_recall'])
